In [1]:
import numpy as np
import pandas as pd
import random

In [2]:
file_path = '/Users/mymac/Google Drive/My Drive/Forex_Robot/'

In [3]:
currency_pair = 'Eur_Usd'
rounding = 3 if 'Jpy' in currency_pair else 5
pips_multiplier = 100 if 'Jpy' in currency_pair else 10000
year_range = '2021-2022'
fractal_window = 5

In [4]:
df = pd.read_csv(file_path + f'Oanda_{currency_pair}_M5_{year_range}.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

In [5]:
def adx(high, low, close, lookback=14):
    plus_dm = high.diff()
    minus_dm = low.diff()
    plus_dm[plus_dm < 0] = 0
    minus_dm[minus_dm > 0] = 0
    
    tr1 = pd.DataFrame(high - low)
    tr2 = pd.DataFrame(abs(high - close.shift(1)))
    tr3 = pd.DataFrame(abs(low - close.shift(1)))
    frames = [tr1, tr2, tr3]
    tr = pd.concat(frames, axis = 1, join = 'inner').max(axis = 1)
    atr = tr.rolling(lookback).mean()
    
    plus_di = 100 * (plus_dm.ewm(alpha = 1/lookback).mean() / atr)
    minus_di = abs(100 * (minus_dm.ewm(alpha = 1/lookback).mean() / atr))
    dx = (abs(plus_di - minus_di) / abs(plus_di + minus_di)) * 100
    adx = ((dx.shift(1) * (lookback - 1)) + dx) / lookback
    adx_smooth = adx.ewm(alpha = 1/lookback).mean()

    return adx_smooth

def stoch(high, low, close, lookback=14):
    high_lookback = high.rolling(lookback).max()
    low_lookback = low.rolling(lookback).min()
    slow_k = (close - low_lookback) * 100 / (high_lookback - low_lookback)
    slow_d = slow_k.rolling(3).mean()

    return slow_k, slow_d

def chop(df, lookback=14):
    atr1 = atr(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], lookback=1)
    high, low = df['Mid_High'], df['Mid_Low']

    chop = np.log10(atr1.rolling(lookback).sum() / (high.rolling(lookback).max() - low.rolling(lookback).min())) / np.log10(lookback)

    return chop

def vo(volume, short_lookback=5, long_lookback=10):
    short_ema =  pd.Series.ewm(volume, span=short_lookback).mean()
    long_ema = pd.Series.ewm(volume, span=long_lookback).mean()

    volume_oscillator = (short_ema - long_ema) / long_ema

    return volume_oscillator

def williams_r(highs, lows, closes, length=21, ema_length=15):
    highest_highs = highs.rolling(window=length).max()
    lowest_lows = lows.rolling(window=length).min()

    willy = 100 * (closes - highest_highs) / (highest_highs - lowest_lows)
    willy_ema = pd.Series.ewm(willy, span=ema_length).mean()

    return willy, willy_ema

def squeeze(barsdata, length=20, length_kc=20, mult=1.5):
    # Bollinger bands
    m_avg = barsdata['Mid_Close'].rolling(window=length).mean()
    m_std = barsdata['Mid_Close'].rolling(window=length).std(ddof=0)
    upper_bb = m_avg + mult * m_std
    lower_bb = m_avg - mult * m_std

    # Keltner channel
    tr0 = abs(barsdata['Mid_High'] - barsdata['Mid_Low'])
    tr1 = abs(barsdata['Mid_High'] - barsdata['Mid_Close'].shift())
    tr2 = abs(barsdata['Mid_Low'] - barsdata['Mid_Close'].shift())
    tr = pd.concat([tr0, tr1, tr2], axis=1).max(axis=1)
    range_ma = tr.rolling(window=length_kc).mean()
    upper_kc = m_avg + range_ma * mult
    lower_kc = m_avg - range_ma * mult

    # Squeeze
    squeeze_on = (lower_bb > lower_kc) & (upper_bb < upper_kc)

    return squeeze_on

def atr(high, low, close, lookback=14):
    high_low = high - low
    high_close = np.abs(high - close.shift())
    low_close = np.abs(low - close.shift())
    ranges = pd.concat([high_low, high_close, low_close], axis=1)
    true_range = np.max(ranges, axis=1)

    return true_range.rolling(lookback).mean()

def atr_bands(high, low, close, lookback=14, atr_multiplier=3):
    scaled_atr_vals = atr(high, low, close, lookback) * atr_multiplier
    lower_band = close - scaled_atr_vals
    upper_band = close + scaled_atr_vals

    return lower_band, upper_band

def rsi(closes, periods=14):
    close_delta = closes.diff()

    up = close_delta.clip(lower=0)
    down = -1 * close_delta.clip(upper=0)
    ma_up = up.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
    ma_down = down.ewm(com = periods - 1, adjust=True, min_periods = periods).mean()
        
    rsi = ma_up / ma_down
    rsi = 100 - (100 / (1 + rsi))

    return rsi

def qqe_mod(closes, rsi_period=6, smoothing=5, qqe_factor=3, threshold=3, mult=0.35, sma_length=50):
    Rsi = rsi(closes, rsi_period)
    RsiMa = Rsi.ewm(span=smoothing).mean()
    AtrRsi = np.abs(RsiMa.shift(1) - RsiMa)
    Wilders_Period = rsi_period * 2 - 1
    MaAtrRsi = AtrRsi.ewm(span=Wilders_Period).mean()
    dar = MaAtrRsi.ewm(span=Wilders_Period).mean() * qqe_factor

    longband = pd.Series(0.0, index=Rsi.index)
    shortband = pd.Series(0.0, index=Rsi.index)
    trend = pd.Series(0, index=Rsi.index)

    DeltaFastAtrRsi = dar
    RSIndex = RsiMa
    newshortband = RSIndex + DeltaFastAtrRsi
    newlongband = RSIndex - DeltaFastAtrRsi
    longband = pd.Series(np.where((RSIndex.shift(1) > longband.shift(1)) & (RSIndex > longband.shift(1)),
                        np.maximum(longband.shift(1), newlongband), newlongband))
    shortband = pd.Series(np.where((RSIndex.shift(1) < shortband.shift(1)) & (RSIndex < shortband.shift(1)),
                        np.minimum(shortband.shift(1), newshortband), newshortband))
    cross_1 = (longband.shift(1) < RSIndex) & (longband > RSIndex)
    cross_2 = (RSIndex > shortband.shift(1)) & (RSIndex.shift(1) < shortband)
    trend = np.where(cross_2, 1, np.where(cross_1, -1, trend.shift(1).fillna(1)))
    FastAtrRsiTL = pd.Series(np.where(trend == 1, longband, shortband))

    basis = (FastAtrRsiTL - 50).rolling(sma_length).mean()
    dev = mult * (FastAtrRsiTL - 50).rolling(sma_length).std()
    upper = basis + dev
    lower = basis - dev

    Greenbar1 = RsiMa - 50 > threshold
    Greenbar2 = RsiMa - 50 > upper

    Redbar1 = RsiMa - 50 < 0 - threshold
    Redbar2 = RsiMa - 50 < lower

    Greenbar = Greenbar1 & Greenbar2
    Redbar = Redbar1 & Redbar2

    return Greenbar, Redbar, RsiMa - 50

def heikin_ashi(open_values, high_values, low_values, close_values):
    ha_close = (open_values + high_values + low_values + close_values) / 4

    ha_open = pd.Series(0.0, index=open_values.index)
    ha_open.iloc[0] = open_values.iloc[0]

    for i in range(1, len(open_values)):
        ha_open.iloc[i] = (ha_open.iloc[i - 1] + ha_close.iloc[i - 1]) / 2

    ha_high = pd.concat([ha_open, ha_close, high_values], axis=1).max(axis=1)
    ha_low = pd.concat([ha_open, ha_close, low_values], axis=1).min(axis=1)

    return ha_open, ha_high, ha_low, ha_close

def trend_indicator(opens, highs, lows, closes, ema_period=50, smoothing_period=10):
    ha_open, _, _, ha_close = heikin_ashi(opens, highs, lows, closes)

    ha_o_ema = pd.Series.ewm(ha_open, span=ema_period).mean()
    ha_c_ema = pd.Series.ewm(ha_close, span=ema_period).mean()

    ha_o_ema_smooth = pd.Series.ewm(ha_o_ema, span=smoothing_period).mean()
    ha_c_ema_smooth = pd.Series.ewm(ha_c_ema, span=smoothing_period).mean()

    return ha_c_ema_smooth > ha_o_ema_smooth

def supertrend(barsdata, atr_len=10, mult=3):
    curr_atr = atr(barsdata['Mid_High'], barsdata['Mid_Low'], barsdata['Mid_Close'], lookback=atr_len)
    highs, lows = barsdata['Mid_High'], barsdata['Mid_Low']
    hl2 = (highs + lows) / 2
    final_upperband = hl2 + mult * curr_atr
    final_lowerband = hl2 - mult * curr_atr

    # initialize Supertrend column to True
    supertrend = [True] * len(df)

    close = barsdata['Mid_Close']
    
    for i in range(1, len(df.index)):
        curr, prev = i, i - 1
        
        # if current close price crosses above upperband
        if close[curr] > final_upperband[prev]:
            supertrend[curr] = True

        # if current close price crosses below lowerband
        elif close[curr] < final_lowerband[prev]:
            supertrend[curr] = False

        # else, the trend continues
        else:
            supertrend[curr] = supertrend[prev]
            
            # adjustment to the final bands
            if supertrend[curr] == True and final_lowerband[curr] < final_lowerband[prev]:
                final_lowerband[curr] = final_lowerband[prev]

            if supertrend[curr] == False and final_upperband[curr] > final_upperband[prev]:
                final_upperband[curr] = final_upperband[prev]

    return supertrend, final_upperband, final_lowerband

def fractal(lows, highs, window=10):
    supports = lows.rolling(window=window, center=True).min()
    resistances = highs.rolling(window=window, center=True).max()

    return supports, resistances

In [6]:
df['ema200'] = pd.Series.ewm(df['Mid_Close'], span=200).mean()
df['qqe_up'], df['qqe_down'], df['qqe_val'] = qqe_mod(df['Mid_Close'])
df['supertrend'], df['supertrend_ub'], df['supertrend_lb'] = supertrend(df)
df['support'], df['resistance'] = fractal(df['Mid_Low'], df['Mid_High'], window=fractal_window)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [7]:
value_per_pip = 1.0
amounts_per_day = [-0.008, -0.01, -0.012] if 'Jpy' in currency_pair else [-0.00008, -0.0001, -0.00012]

In [8]:
def get_n_units(trade_type, stop_loss, ask_open, bid_open, mid_open, currency_pair):
    _, second = currency_pair.split('_')
  
    pips_to_risk = ask_open - stop_loss if trade_type == 'buy' else stop_loss - bid_open
    pips_to_risk_calc = pips_to_risk * 10000 if second != 'Jpy' else pips_to_risk * 100

    if second == 'Usd':
        per_pip = 0.0001

    else:
        per_pip = 0.0001 / mid_open if second != 'Jpy' else 0.01 / mid_open

    n_units = int(50 / (pips_to_risk_calc * per_pip))

    return n_units

def calculate_day_fees(start_date, end_date, n_units):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25]) * n_units
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee

def run_simulation(spread_cutoff, use_supertrend_pullback):
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked, win_amounts, loss_amounts = [], [], []
    day_fees = 0
    trade = None

    for i in range(2, len(df)):
        curr_date = df.loc[df.index[i], 'Date']
        curr_ao, curr_bo, curr_mid_open, curr_ask_low, curr_bid_high, curr_bid_low, curr_ask_high = df.loc[df.index[i], ['Ask_Open', 'Bid_Open', 'Mid_Open', 'Ask_Low', 'Bid_High', 'Bid_Low', 'Ask_High']]
        spread = abs(curr_ao - curr_bo)

        if trade is None:
            mid_open1, mid_close1, mid_low1, mid_high1 = df.loc[df.index[i - 1], ['Mid_Open', 'Mid_Close', 'Mid_Low', 'Mid_High']]

            supertrend2 = df.loc[df.index[i - 2], 'supertrend']
            supertrend1, supertrend_ub, supertrend_lb, qqe_up, qqe_down, ema200 = df.loc[df.index[i - 1], ['supertrend', 'supertrend_ub', 'supertrend_lb', 'qqe_up', 'qqe_down', 'ema200']]

            supertrend_buy_signal = not supertrend2 and supertrend1
            supertrend_sell_signal = supertrend2 and not supertrend1

            if supertrend_buy_signal and qqe_up and mid_close1 > ema200:
            # if supertrend_buy_signal and qqe_up:
                open_price = float(curr_ao)
                pullback = float(supertrend_lb) - spread

                stop_loss = round(pullback, rounding)

                if stop_loss < open_price:
                    curr_pips_to_risk = open_price - stop_loss

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        n_units = get_n_units('buy', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                         'pips_risked': round(curr_pips_to_risk, 5),
                                                        'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_buys += 1

            elif supertrend_sell_signal and qqe_down and mid_close1 < ema200:
            # elif supertrend_sell_signal and qqe_down:
                open_price = float(curr_bo)
                pullback = float(supertrend_ub) + spread

                stop_loss = round(pullback, rounding)

                if stop_loss > open_price:
                    curr_pips_to_risk = stop_loss - open_price

                    if spread <= curr_pips_to_risk * spread_cutoff:
                        n_units = get_n_units('sell', stop_loss, curr_ao, curr_bo, curr_mid_open, currency_pair)

                        trade = {'start_index': i, 'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                 'pips_risked': round(curr_pips_to_risk, 5),
                                'n_units': n_units, 'original_units': n_units, 'start_date': curr_date, 'end_date': None}
                        
                        pips_risked.append(curr_pips_to_risk)
                        n_sells += 1

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'buy' and curr_bid_high > trade['open_price']:
            if use_supertrend_pullback:
                supertrend2 = df.loc[df.index[i - 1], 'supertrend']
                supertrend1, supertrend_lb = df.loc[df.index[i], ['supertrend', 'supertrend_lb']]

                if not supertrend2 and supertrend1 and supertrend_lb > min(trade['open_price'], trade['stop_loss']):
                   trade['stop_loss'] = round(float(supertrend_lb) - spread, rounding)

            else:
                support = df.loc[df.index[i], 'support']

                if support > min(trade['open_price'], trade['stop_loss']):
                    trade['stop_loss'] = round(float(support) - spread, rounding)


        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            # trade_amount = -50.0
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date, trade['n_units'])

            if trade_amount > 0:
                win_amounts.append(trade_amount)

            else:
                loss_amounts.append(trade_amount)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade = None

        if trade is not None and trade['trade_type'] == 'sell' and curr_ask_low < trade['open_price']:
            if use_supertrend_pullback:
                supertrend2 = df.loc[df.index[i - 1], 'supertrend']
                supertrend1, supertrend_ub = df.loc[df.index[i], ['supertrend', 'supertrend_ub']]

                if supertrend2 and not supertrend1 and supertrend_ub < max(trade['open_price'], trade['stop_loss']):
                   trade['stop_loss'] = round(float(supertrend_ub) + spread, rounding)

            else:
                resistance = df.loc[df.index[i], 'resistance']

                if resistance < max(trade['open_price'], trade['stop_loss']):
                    trade['stop_loss'] = round(float(resistance) + spread, rounding)

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked, win_amounts, loss_amounts

In [11]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
spread_cutoffs = [0.05, 0.1]
use_supertrend_pullback_vals = [True, False]

all_combos = []

for spread_cutoff in spread_cutoffs:
    for use_supertrend_pullback in use_supertrend_pullback_vals:
        all_combos.append((spread_cutoff, use_supertrend_pullback))

percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_spread_cutoff = None
best_use_supertrend_pullback = None
top_n_results = 10
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for spread_cutoff, use_supertrend_pullback in combos_to_try:
    reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak, pips_risked,  win_amounts, loss_amounts = run_simulation(spread_cutoff, use_supertrend_pullback)
    runs_finished += 1

    print(reward)
    print('Num buys: ' + str(n_sells))
    print('Num sells: ' + str(n_buys))
    print('Num trades: ' + str(n_buys + n_sells))
    print('Num wins: ' + str(n_wins))
    print('Num losses: ' + str(n_losses))
    print('Win streak: ' + str(win_streak))
    print('Loss streak: ' + str(loss_streak))
    if len(pips_risked) > 0:
        print('Avg pips risked: ' + str(np.array(pips_risked).mean()))
    if len(win_amounts) > 0:
        print('Avg win amount: ' + str(np.array(win_amounts).mean()))
        print(min(win_amounts))
        print(max(win_amounts))
    if len(loss_amounts) > 0:
        print('Avg loss amount: ' + str(np.array(loss_amounts).mean()))
        print(min(loss_amounts))
        print(max(loss_amounts))

    print('Remaining runs: ' + str(n_runs - runs_finished))

    min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None

    if min_item is None or reward > min_item['reward']:
        if min_item is not None:
            best_rewards.remove(min_item)
            
        best_rewards.append({'reward': int(reward), 'spread_cutoff': spread_cutoff, 'use_supertrend_pullback': use_supertrend_pullback})


    if reward > best_reward:
        best_reward = reward
        best_spread_cutoff = spread_cutoff
        best_use_supertrend_pullback = use_supertrend_pullback


    print('Best reward so far: ' + str(best_reward))
    print()

Num runs: 4

-1905.2485099999624
Num buys: 180
Num sells: 78
Num trades: 258
Num wins: 77
Num losses: 180
Win streak: 4
Loss streak: 9
Avg pips risked: 0.002092558139534885
Avg win amount: 82.4162511688315
0.23147999999894653
318.9499200000033
Avg loss amount: -44.40425138888881
-49.999999999998934
-0.9478400000009479
Remaining runs: 3
Best reward so far: -1905.2485099999624

53.93658999998788
Num buys: 27
Num sells: 17
Num trades: 44
Num wins: 15
Num losses: 29
Win streak: 3
Loss streak: 6
Avg pips risked: 0.0036375000000000105
Avg win amount: 37.39474733333308
0.23639999999892414
100.15697999999996
Avg loss amount: -17.24102551724167
-49.99995000000083
-0.11574000000075824
Remaining runs: 2
Best reward so far: 53.93658999998788

-1467.559480000028
Num buys: 230
Num sells: 102
Num trades: 332
Num wins: 82
Num losses: 245
Win streak: 3
Loss streak: 16
Avg pips risked: 0.0020439759036144627
Avg win amount: 40.35385024390218
0.23639999999892414
258.8150400000031
Avg loss amount: -18.9169

In [12]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best spread cutoff: ' + str(best_spread_cutoff))
print('Best use supertrend as pullback val: ' + str(use_supertrend_pullback))
print('-----------------------')
print('Top results:')

for entry in best_rewards:
    print(entry)

------------ FINAL RESULTS ------------
Best reward: 53.93658999998788
Best spread cutoff: 0.05
Best use supertrend as pullback val: True
-----------------------
Top results:
{'reward': -1905, 'spread_cutoff': 0.1, 'use_supertrend_pullback': True}
{'reward': 53, 'spread_cutoff': 0.05, 'use_supertrend_pullback': False}
{'reward': -1467, 'spread_cutoff': 0.1, 'use_supertrend_pullback': False}
{'reward': -133, 'spread_cutoff': 0.05, 'use_supertrend_pullback': True}
